In [1]:
# Reload automatically when the file is changed.
%load_ext autoreload
%autoreload 2

# Importing Modules

In [2]:
import os 
import sys
MODULES_PATH = "../../Modules/RNN"
MODELS_PATH = "../../Models"

sys.path.append(MODULES_PATH)
sys.path.append(MODELS_PATH)


from Preprocessing import *
from Dataloaders_Preprocessing import *
from Train import *
from RNN import *






# Importing Data

In [3]:
List_Data_Train = Load_Data('../../Data/atoms/train', '../../Data/energies/train.csv')
List_Data_Test= Load_Test_Data('../../Data/atoms/test')

In [4]:
Display_Molecule_From_Atom_List(List_Data_Train[0]['Atoms_List'], Width=800, Height=800, Background_Color='lightblue')


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:
# Dataset_Train = MoleculeDataset_Transformer(Data_List = List_Data_Train,
#                                             Nb_Atoms_Max_In_Molecule=23, Return_Energies=True, transform=None)

# Dataset_Val = MoleculeDataset_Transformer(Data_List = List_Data_Val,
#                                             Nb_Atoms_Max_In_Molecule=23, Return_Energies=False, transform=None)

Dataset_Train = MoleculeDataset_LSTM(Data_List = List_Data_Train, Return_Energies=True, transform=None)

Dataset_Test = MoleculeDataset_LSTM(Data_List = List_Data_Test, Return_Energies=False, transform=None)


In [13]:
import torch
import torch.nn as nn
from RNN import MoleculeLSTM  # Assurez-vous que le fichier RNN.py est accessible

# Configuration des paramètres pour la classe MoleculeLSTM
NUM_ATOM_TYPES = 23
EMBEDDING_DIM = 512
LSTM_HIDDEN_DIM = 512
LSTM_NUM_LAYERS = 3
NB_HIDDEN_LAYERS_MLP = 5
HIDDEN_LAYERS_SIZE_LIST_MLP = [512, 256, 128, 64, 32]  # Liste des tailles des couches cachées pour le MLP
OUTPUT_DIM = 1
ACTIVATION_NAME_MLP = 'PReLU'

# Affichage des paramètres pour vérification
print("Configuration des paramètres du MoleculeLSTM :")
print(f"NUM_ATOM_TYPES: {NUM_ATOM_TYPES}")
print(f"EMBEDDING_DIM: {EMBEDDING_DIM}")
print(f"LSTM_HIDDEN_DIM: {LSTM_HIDDEN_DIM}")
print(f"LSTM_NUM_LAYERS: {LSTM_NUM_LAYERS}")
print(f"NB_HIDDEN_LAYERS_MLP: {NB_HIDDEN_LAYERS_MLP}")
print(f"HIDDEN_LAYERS_SIZE_LIST_MLP: {HIDDEN_LAYERS_SIZE_LIST_MLP}")
print(f"OUTPUT_DIM: {OUTPUT_DIM}")
print(f"ACTIVATION_NAME_MLP: {ACTIVATION_NAME_MLP}")

# Instanciation du modèle
Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {Device}")

Model = MoleculeLSTM(
    Num_Atom_Types=NUM_ATOM_TYPES,
    Embedding_Dim=EMBEDDING_DIM,
    LSTM_Hidden_Dim=LSTM_HIDDEN_DIM,
    LSTM_Num_Layers=LSTM_NUM_LAYERS,
    Nb_Hidden_Layers_MLP=NB_HIDDEN_LAYERS_MLP,
    Hidden_Layers_Size_List_MLP=HIDDEN_LAYERS_SIZE_LIST_MLP,
    Output_Dim=OUTPUT_DIM,
    Activation_Name_MLP=ACTIVATION_NAME_MLP
)

Model.to(Device)
print("Modèle MoleculeLSTM instancié avec succès.")

Configuration des paramètres du MoleculeLSTM :
NUM_ATOM_TYPES: 23
EMBEDDING_DIM: 512
LSTM_HIDDEN_DIM: 512
LSTM_NUM_LAYERS: 3
NB_HIDDEN_LAYERS_MLP: 5
HIDDEN_LAYERS_SIZE_LIST_MLP: [512, 256, 128, 64, 32]
OUTPUT_DIM: 1
ACTIVATION_NAME_MLP: PReLU
Using device: cuda
Modèle MoleculeLSTM instancié avec succès.


In [14]:
PATH_SAVING_MODEL = '../../Saved_Models/RNN/MoleculeLSTM.pth'

# Model = torch.load(PATH_SAVING_MODEL)   
# Params_Model = Model.state_dict()

In [15]:
# NB parameters
Nb_Parameters = sum(p.numel() for p in Model.parameters() if p.requires_grad)
print(f"Number of parameters in the model: {Nb_Parameters}")

Number of parameters in the model: 6758918


In [16]:
DataLoader_Train = Create_DataLoader_LSTM(List_Data_Train, Batch_Size=128, Shuffle=True, Num_Workers=0, Test_Size=0, Return_Energies=True,Transform=None)
DataLoader_Test = Create_DataLoader_LSTM(List_Data_Test, Batch_Size=64, Shuffle=False, Num_Workers=0, Test_Size=0, Return_Energies=False, Transform=None)

In [17]:
import torch.optim as optim
import torch.nn as nn

In [18]:

Optimizer = torch.optim.Adam(Model.parameters(), lr=3e-4)
Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Criterion = nn.MSELoss()

List_Train_Losses_Per_Epochs, List_Test_Losses_Per_Epochs, List_Train_Losses_Per_Batches, List_Test_Losses_Per_Batches = Train(
    Model,
    DataLoader_Train,
    DataLoader_Train,
    Optimizer,
    Criterion,
    Num_Epochs=450,
    Device=Device,
    Save_Path=PATH_SAVING_MODEL
)

Plot_Losses(
    List_Train_Losses_Per_Epochs,
    List_Test_Losses_Per_Epochs,
    List_Train_Losses_Per_Batches,
    List_Test_Losses_Per_Batches,
    Save=True,
    Save_Path='losses_plot.png'
)





Epochs:   0%|          | 0/450 [00:00<?, ?it/s]

Batches:   0%|          | 0/52 [00:00<?, ?it/s]

Batches:   0%|          | 0/52 [00:00<?, ?it/s]

New best test loss: 347.121, model saved to ../../Saved_Models/RNN/MoleculeLSTM.pth
Epoch: 1/450 Train Loss: 1883.869 Test Loss: 347.121 


Batches:   0%|          | 0/52 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/52 [00:00<?, ?it/s]

Batches:   0%|          | 0/52 [00:00<?, ?it/s]

New best test loss: 304.313, model saved to ../../Saved_Models/RNN/MoleculeLSTM.pth
Epoch: 5/450 Train Loss: 304.535 Test Loss: 304.313 


Batches:   0%|          | 0/52 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
os.makedirs('./Saved_Models/Transformer', exist_ok=False)

In [ ]:
PATH_SAVING_MODEL = '../Saved_Models/Transformers/Transformer_Atom_Energy_Prediction.pth'
torch.save(Model, PATH_SAVING_MODEL)
print(f"Model saved to {PATH_SAVING_MODEL}")

In [12]:
# Load weights 

Model_Imported = torch.load(PATH_SAVING_MODEL)

/tmp/ipykernel_194780/1153814197.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Model_Imported = torch.load(PATH_SAVING_MODEL)


In [13]:
Model_Imported

Transformer(
  (atom_embedding): Embedding(6, 512)
  (position_projection): Linear(in_features=3, out_features=512, bias=True)
  (attention_blocks): ModuleList(
    (0): ModuleDict(
      (attention): Multi_Head_Attention(
        (WQ): ModuleList(
          (0-4): 5 x MLP(
            (model): Sequential(
              (0): Linear(in_features=1024, out_features=2048, bias=True)
              (1): PReLU(num_parameters=1)
              (2): Linear(in_features=2048, out_features=1024, bias=True)
              (3): PReLU(num_parameters=1)
              (4): Linear(in_features=1024, out_features=512, bias=True)
              (5): PReLU(num_parameters=1)
              (6): Linear(in_features=512, out_features=512, bias=True)
            )
          )
        )
        (WK): ModuleList(
          (0-4): 5 x MLP(
            (model): Sequential(
              (0): Linear(in_features=1024, out_features=2048, bias=True)
              (1): PReLU(num_parameters=1)
              (2): Linear(in_fea

In [14]:
Dataset_Test[0]

(7400,
 tensor([1, 1, 1, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([[-0.4119, -0.2555,  0.5291],
         [-0.9582,  0.6230, -0.5784],
         [ 1.0879, -0.4802,  0.5894],
         [-0.5571,  1.6376, -0.4632],
         [ 1.3639, -1.2178, -0.1985],
         [-0.6270,  0.1140, -1.9801],
         [ 0.4483,  0.0663, -2.1417],
         [ 1.6701,  1.2024, -0.4485],
         [ 1.8555,  0.7610,  0.4485],
         [-1.2310, -0.8438,  1.4115],
         [-2.0441,  0.7062, -0.4710],
         [ 1.3516, -0.9488,  1.5470],
         [-1.0320, -0.8827, -2.1403],
         [-2.3056, -0.7173,  1.3544],
         [-0.8450, -1.4613,  2.2141],
         [-1.0498,  0.7734, -2.7356],
         [ 2.8519,  0.5700,  0.5145],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]]),
 tensor([1., 1., 1., 1., 1., 1., 1., 

In [15]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

def Compute_Predictions(Model, Dataset_Test, Device, batch_size=32):
    """
    Computes predictions on the test dataset using the Transformer model.

    Args:
        Model: The Transformer model.
        Dataset_Test: MoleculeDataset_Transformer instance for the test set.
        Device: torch.device (e.g., 'cuda:0' or 'cpu').
        batch_size: Batch size for processing (default: 32).

    Returns:
        Ids: List of molecule identifiers.
        Predictions: List of predicted energy values (scalars).
    """
    Model.eval()  # Set the model to evaluation mode
    Model.to(Device)

    # Create DataLoader for efficient batch processing
    test_loader = DataLoader(Dataset_Test, batch_size=batch_size, shuffle=False)

    Predictions = []
    Ids = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Computing Predictions"):
            Ids_batch, symbols, positions, mask = batch  # Ignore energy
            symbols = symbols.to(Device)
            positions = positions.to(Device)
            mask = mask.to(Device)

            # Forward pass
            outputs = Model(symbols, positions, mask=mask)  # Shape: (batch_size, 1)
            outputs = outputs.squeeze().cpu().tolist()  # Convert to list of scalars

            # Extend lists
            Ids.extend(Ids_batch)
            if isinstance(outputs, float):  # Single prediction case
                Predictions.append(outputs)
            else:  # Batch of predictions
                Predictions.extend(outputs)


    # transform ids to list of ints
    Ids = [int(id) for id in Ids]
    print(Ids)
    return Ids, Predictions

In [16]:
Ids, Predictions = Compute_Predictions(Model, Dataset_Test, Device)

Computing Predictions: 100%|██████████| 52/52 [00:03<00:00, 13.53it/s]

[7400, 7691, 7584, 7356, 8122, 7594, 7654, 7972, 7069, 7149, 7656, 7123, 7762, 6752, 7212, 7830, 8154, 7155, 7042, 7007, 7625, 7898, 8171, 7627, 6853, 6979, 7120, 7079, 7865, 7489, 7140, 7086, 6769, 7750, 7049, 7809, 6804, 7052, 7991, 6620, 8204, 7421, 6745, 6614, 7497, 8215, 7590, 8219, 7857, 8225, 7430, 8094, 6915, 7814, 6836, 7790, 7264, 6872, 7452, 7144, 6898, 8110, 7682, 7667, 8202, 7797, 6690, 6615, 7617, 8120, 6714, 7056, 7288, 7258, 7646, 7027, 8034, 8187, 7328, 7365, 7649, 7015, 7628, 8228, 7285, 7274, 7554, 7588, 6838, 6749, 6935, 6940, 8124, 7455, 7779, 7689, 6727, 7770, 7729, 8231, 6709, 7199, 7166, 7343, 6826, 7701, 7521, 8064, 7088, 7501, 7366, 7085, 7676, 6675, 6831, 6818, 6652, 7512, 8190, 6842, 6688, 7329, 7593, 6683, 8126, 6685, 7029, 7752, 7292, 6765, 6968, 7672, 7075, 6904, 7153, 7347, 7640, 8097, 7192, 8035, 7828, 7299, 8174, 6862, 7003, 6887, 8076, 7981, 7262, 6787, 7964, 7437, 6931, 7575, 8008, 7558, 7996, 7287, 7848, 7239, 7414, 7019, 8189, 6892, 7146, 7949, 819

In [17]:
Predictions

[-76.06282043457031,
 -66.8908920288086,
 -72.35344696044922,
 -74.9702377319336,
 -90.25334167480469,
 -97.53258514404297,
 -77.60855102539062,
 -103.34635925292969,
 -83.7620849609375,
 -90.36339569091797,
 -74.40926361083984,
 -84.30471801757812,
 -84.49021911621094,
 -64.14910125732422,
 -76.317626953125,
 -74.9131851196289,
 -82.85950469970703,
 -69.6265640258789,
 -91.08336639404297,
 -69.40583801269531,
 -83.52899932861328,
 -97.18384552001953,
 -75.0822982788086,
 -90.35279846191406,
 -76.43041229248047,
 -84.83358001708984,
 -72.18324279785156,
 -77.0469970703125,
 -72.91141510009766,
 -68.6390151977539,
 -83.79907989501953,
 -65.37931823730469,
 -83.39419555664062,
 -63.230560302734375,
 -69.38199615478516,
 -56.693199157714844,
 -77.45246887207031,
 -91.0719223022461,
 -83.77436828613281,
 -77.22904205322266,
 -76.00826263427734,
 -90.49967193603516,
 -69.21595764160156,
 -90.66510772705078,
 -97.18270111083984,
 -78.07874298095703,
 -83.96807098388672,
 -82.33061981201172,


In [18]:
# create a csv file with the predictions

import pandas as pd
def Save_Predictions_To_CSV(Ids, Predictions, Output_CSV_Path):
    df = pd.DataFrame({
        'id': Ids,
        'energy': Predictions
    })
        # sort the dataframe by id
    df.sort_values(by='id', inplace=True)
    df.to_csv(Output_CSV_Path, index=False)


    print(f"Predictions saved to {Output_CSV_Path}")

Save_Predictions_To_CSV(Ids, Predictions, 'predictions.csv')

Predictions saved to predictions.csv


In [ ]:
import numpy as np




def test_rotation_invariance():
    # Crée une "molécule" avec 3 atomes
    original_positions = np.array([
        [0.0, 0.0, 0.0],
        [5.0, 0.0, 0.0],
        [0.0, 1.0, 0.0]
    ])

    transformed_positions = random_translate_and_rotate(original_positions, max_translation=5.0)

    print("Original:\n", original_positions)
    print("Transformé:\n", transformed_positions)

    # Vérification des distances inter-atomiques (doivent rester les mêmes)
    def pairwise_distances(positions):
        n = len(positions)
        distances = []
        for i in range(n):
            for j in range(i + 1, n):
                d = np.linalg.norm(positions[i] - positions[j])
                distances.append(d)
        return np.array(distances)

    d_orig = pairwise_distances(original_positions)
    d_trans = pairwise_distances(transformed_positions)

    print("\nDistances originales:", d_orig)
    print("Distances transformées:", d_trans)

    if np.allclose(d_orig, d_trans, atol=1e-5):
        print("\n✅ La rotation conserve les distances : test réussi.")
    else:
        print("\n❌ Les distances ont changé : problème avec la rotation.")


    # Plot les positions originales et transformées
    import matplotlib.pyplot as plt
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(121, projection='3d')
    ax.scatter(original_positions[:, 0], original_positions[:, 1], original_positions[:, 2], color='blue', label='Original')
    ax.set_title('Positions Originales')
    ax = fig.add_subplot(122, projection='3d')
    ax.scatter(transformed_positions[:, 0], transformed_positions[:, 1], transformed_positions[:, 2], color='red', label='Transformé')
    ax.set_title('Positions Transformées')
    plt.show()
    

# Lancer le test
test_rotation_invariance()


## XgBoost 


In [9]:
PATH_SAVING_MODEL = '../Saved_Models/Transformers/V1/Transformer_Atom_Energy_Prediction.pth'

In [10]:
Model = torch.load(PATH_SAVING_MODEL)

/tmp/ipykernel_9942/2750430278.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Model = torch.load(PATH_SAVING_MODEL)


In [11]:
# Récupérer les CLS Tokens piur le train : 
Model.to(Device)
Ids_Train = []
Cls_Tokens_Train = []
Energies = []
Progress_Bar = tqdm(total=len(Dataset_Train), desc="Calcul des CLS Tokens pour l'entraînement")
for data in Dataset_Train:
    Ids, symbols, positions, mask, energies = data
    symbols = symbols.to(Device).unsqueeze(0)  # Assurez-vous que symbols est de la bonne forme
    positions = positions.to(Device).unsqueeze(0)  # Assurez-vous que positions est de la bonne forme
    mask = mask.to(Device).unsqueeze(0)  # Assurez-vous que mask est de la bonne forme
    output, cls_token = Model.forward(symbols, positions, mask=mask, return_cls_token=True)
    Cls_Tokens_Train.append(cls_token.cpu().detach().numpy())
    Energies.append(energies.cpu().detach().numpy())
    Ids_Train.append(Ids)
    Progress_Bar.update(1)
Progress_Bar.close()



Calcul des CLS Tokens pour l'entraînement:   0%|          | 0/6591 [00:00<?, ?it/s]

In [12]:
Cls_Token_Test = []
Progress_Bar = tqdm(total=len(Dataset_Test), desc="Calcul des CLS Tokens pour le test")
Ids_Test = []
for data in Dataset_Test:
    Ids, symbols, positions, mask = data
    symbols = symbols.to(Device).unsqueeze(0)  # Assurez-vous que symbols est de la bonne forme
    positions = positions.to(Device).unsqueeze(0)  # Assurez-vous que positions est de la bonne forme
    mask = mask.to(Device).unsqueeze(0)  # Assurez-vous que mask est de la bonne forme
    output, cls_token = Model.forward(symbols, positions, mask=mask, return_cls_token=True)
    Cls_Token_Test.append(cls_token.cpu().detach().numpy())
    Ids_Test.append(Ids)
    Progress_Bar.update(1)
Progress_Bar.close()
# Convertir les listes en tableaux numpy
Cls_Tokens_Train = np.array(Cls_Tokens_Train).squeeze()

Calcul des CLS Tokens pour le test:   0%|          | 0/1647 [00:00<?, ?it/s]

In [13]:


Cls_Tokens_Train,Energies = np.array(Cls_Tokens_Train), np.array(Energies)


In [14]:
import os 
import sys
MODULES_PATH = "../Modules"
MODELS_PATH = "../Models"

sys.path.append(MODULES_PATH)
sys.path.append(MODELS_PATH)


from Preprocessing import *
from Dataloaders_Preprocessing import *
from XG_Boost import *






In [34]:
Model_XgBoost = XGBoostWrapper(
    input_dim=Cls_Tokens_Train.shape[1], n_estimators=10000, max_depth=6, learning_rate=0.1, objective='reg:squarederror', model_type='regressor',
    reg_lambda=0.1, reg_alpha=0.1)

2025-06-19 10:24:24,662 - INFO - XGBoostWrapper initialisé avec input_dim=1024, model_type=regressor, reg_lambda=0.1, reg_alpha=0.1


In [35]:
Model_XgBoost.fit(X = Cls_Tokens_Train, y = Energies, early_stopping_rounds=1000,verbose=True, eval_set=[(Cls_Tokens_Train, Energies)])

2025-06-19 10:24:25,480 - INFO - Early stopping activé avec 1000 rounds


[0]	train-rmse:10.49029	eval-rmse:10.49029
[1]	train-rmse:9.44420	eval-rmse:9.44420
[2]	train-rmse:8.50258	eval-rmse:8.50258
[3]	train-rmse:7.65515	eval-rmse:7.65515
[4]	train-rmse:6.89231	eval-rmse:6.89231
[5]	train-rmse:6.20573	eval-rmse:6.20573
[6]	train-rmse:5.58772	eval-rmse:5.58772
[7]	train-rmse:5.03151	eval-rmse:5.03151
[8]	train-rmse:4.53091	eval-rmse:4.53091
[9]	train-rmse:4.08042	eval-rmse:4.08042
[10]	train-rmse:3.67503	eval-rmse:3.67503
[11]	train-rmse:3.31027	eval-rmse:3.31027
[12]	train-rmse:2.98204	eval-rmse:2.98204
[13]	train-rmse:2.68675	eval-rmse:2.68675
[14]	train-rmse:2.42111	eval-rmse:2.42111
[15]	train-rmse:2.18219	eval-rmse:2.18219
[16]	train-rmse:1.96738	eval-rmse:1.96738
[17]	train-rmse:1.77424	eval-rmse:1.77424
[18]	train-rmse:1.60062	eval-rmse:1.60062
[19]	train-rmse:1.44461	eval-rmse:1.44461
[20]	train-rmse:1.30454	eval-rmse:1.30454
[21]	train-rmse:1.17877	eval-rmse:1.17877
[22]	train-rmse:1.06591	eval-rmse:1.06591
[23]	train-rmse:0.96472	eval-rmse:0.96472


2025-06-19 10:26:07,314 - INFO - Entraînement terminé avec succès


In [36]:
Cls_Token_Test = np.array(Cls_Token_Test).squeeze()

In [37]:
Responses_Test,_ = Model_XgBoost.evaluate(X = Cls_Token_Test)

In [38]:
Responses_Test

[-76.1875228881836,
 -67.01236724853516,
 -72.05703735351562,
 -74.89888763427734,
 -90.31306457519531,
 -97.21377563476562,
 -78.1419677734375,
 -103.2925033569336,
 -83.70619201660156,
 -90.2063217163086,
 -74.43048858642578,
 -84.2356185913086,
 -84.35139465332031,
 -64.24664306640625,
 -76.31099700927734,
 -74.92632293701172,
 -82.93508911132812,
 -69.63052368164062,
 -90.86883544921875,
 -69.30487823486328,
 -83.3850326538086,
 -97.10682678222656,
 -74.99748229980469,
 -90.15196990966797,
 -76.25279998779297,
 -84.92105865478516,
 -72.04585266113281,
 -76.6043930053711,
 -73.09860229492188,
 -68.8283462524414,
 -84.06541442871094,
 -65.33350372314453,
 -83.39966583251953,
 -63.57051086425781,
 -69.24972534179688,
 -56.42507553100586,
 -76.98685455322266,
 -90.8832778930664,
 -83.85832214355469,
 -76.99932098388672,
 -76.17186737060547,
 -90.12763977050781,
 -69.2321548461914,
 -90.52195739746094,
 -97.05474853515625,
 -78.11231231689453,
 -83.97911071777344,
 -82.41221618652344,
 

In [39]:
# create a csv file with the predictions

import pandas as pd
def Save_Predictions_To_CSV(Ids, Predictions, Output_CSV_Path):
    df = pd.DataFrame({
        'id': Ids,
        'energy': Predictions
    })
        # sort the dataframe by id
    df.sort_values(by='id', inplace=True)
    df.to_csv(Output_CSV_Path, index=False)


    print(f"Predictions saved to {Output_CSV_Path}")

Save_Predictions_To_CSV(Ids_Test, Responses_Test, 'predictions_xgboost.csv')

Predictions saved to predictions_xgboost.csv
